In [1]:
from qiskit import QuantumCircuit, execute, transpile
from qiskit_emulator import DellHybridProvider

In [2]:
provider = DellHybridProvider()

In [3]:
provider.remote('http://qre-vf.oro-sandbox-small1.k8s.cec.lab.emc.com')

In [4]:
provider.runtime.programs()

{'4f2b45518fa1': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f927ce798b0>}

In [5]:
RUNTIME_PROGRAM = """
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):

    user_messenger.publish({'results': 'intermittently'})

    circuits = transpile(
        circuits,
    )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    for i in range(0, 10):
        result = backend.run(circuits, **kwargs).result()
        user_messenger.publish({f'result_{i}': result)
    
    user_messenger.publish('completed', final=True)
    print("job complete successfully")
"""

In [6]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [7]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [8]:
program_id

'cb9e10dcb3dd'

In [9]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [10]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs, callback=print)

In [12]:
job.get_unread_messages()

[]

In [13]:
job.result(timeout=20)

TypeError: exceptions must derive from BaseException

In [14]:
job.status()

AttributeError: 'EmulatorRuntimeJob' object has no attribute 'status'